# Import libaries

In [1]:
import openml
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from progressbar import ProgressBar
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
import time
import json

# Import all datatsets

In [2]:
datasets_df = openml.datasets.list_datasets(output_format="dataframe")
datasets_df.head(10)

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
7,7,audiology,1,1,active,ARFF,57.0,24.0,1.0,24.0,70.0,226.0,222.0,317.0,0.0,70.0
8,8,liver-disorders,1,1,active,ARFF,NaN,NaN,NaN,0.0,6.0,345.0,0.0,0.0,6.0,0.0
9,9,autos,1,1,active,ARFF,67.0,22.0,3.0,6.0,26.0,205.0,46.0,59.0,15.0,11.0
10,10,lymph,1,1,active,ARFF,81.0,8.0,2.0,4.0,19.0,148.0,0.0,0.0,3.0,16.0
11,11,balance-scale,1,1,active,ARFF,288.0,3.0,49.0,3.0,5.0,625.0,0.0,0.0,4.0,1.0


# Extract Dataset

In [3]:
dataset = openml.datasets.get_dataset(294)

# Print description about dataset

In [4]:
# Print a summary
print(
    f"This is dataset '{dataset.name}', the target feature is "
    f"'{dataset.default_target_attribute}'"
)
print(f"URL: {dataset.url}")
print(dataset.description[:])

This is dataset 'satellite_image', the target feature is 'class'
URL: https://www.openml.org/data/v1/download/50720/satellite_image.arff
**Author**:   
**Source**: Unknown - 1993  
**Please cite**:   

Source:
Ashwin Srinivasan
Department of Statistics and Data Modeling
University of Strathclyde
Glasgow
Scotland
UK
ross '@' uk.ac.turing

The original Landsat data for this database was generated from data purchased from NASA by the Australian Centre for Remote Sensing, and used for research at: 
The Centre for Remote Sensing
University of New South Wales
Kensington, PO Box 1
NSW 2033
Australia.

The sample database was generated taking a small section (82 rows and 100 columns) from the original data. The binary values were converted to their present ASCII form by Ashwin Srinivasan. The classification for each pixel was performed on the basis of an actual site visit by Ms. Karen Hall, when working for Professor John A. Richards, at the Centre for Remote Sensing at the University of New S

# Get depent and indepent variables

In [5]:
x, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="dataframe", target=dataset.default_target_attribute
)

# Data types

In [6]:
x.dtypes

attr1     uint8
attr2     uint8
attr3     uint8
attr4     uint8
attr5     uint8
attr6     uint8
attr7     uint8
attr8     uint8
attr9     uint8
attr10    uint8
attr11    uint8
attr12    uint8
attr13    uint8
attr14    uint8
attr15    uint8
attr16    uint8
attr17    uint8
attr18    uint8
attr19    uint8
attr20    uint8
attr21    uint8
attr22    uint8
attr23    uint8
attr24    uint8
attr25    uint8
attr26    uint8
attr27    uint8
attr28    uint8
attr29    uint8
attr30    uint8
attr31    uint8
attr32    uint8
attr33    uint8
attr34    uint8
attr35    uint8
attr36    uint8
dtype: object

In [7]:
# cat_columns = x.select_dtypes(['category']).columns
# x[cat_columns] = x[cat_columns].apply(lambda x: pd.factorize(x)[0])

In [8]:
x.dtypes

attr1     uint8
attr2     uint8
attr3     uint8
attr4     uint8
attr5     uint8
attr6     uint8
attr7     uint8
attr8     uint8
attr9     uint8
attr10    uint8
attr11    uint8
attr12    uint8
attr13    uint8
attr14    uint8
attr15    uint8
attr16    uint8
attr17    uint8
attr18    uint8
attr19    uint8
attr20    uint8
attr21    uint8
attr22    uint8
attr23    uint8
attr24    uint8
attr25    uint8
attr26    uint8
attr27    uint8
attr28    uint8
attr29    uint8
attr30    uint8
attr31    uint8
attr32    uint8
attr33    uint8
attr34    uint8
attr35    uint8
attr36    uint8
dtype: object

# Change response variable to int

In [9]:
y

0       3
1       3
2       3
3       3
4       3
       ..
6430    1
6431    1
6432    5
6433    5
6434    5
Name: class, Length: 6435, dtype: uint8

In [10]:
y=y.map({'1' :0,'2':1})

In [11]:
y.astype(int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df_min_max_scaled = x.copy()
  
# apply normalization techniques
for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  
# view normalized data
print(df_min_max_scaled)

In [ ]:
pbar = ProgressBar()
x=df_min_max_scaled.to_numpy()

In [ ]:
x

In [ ]:
DatasetName='Satelite.json'
cross="Satelite_cross.json"

# Grid search cv function

In [ ]:
def gridfun(fName,x,y,estimator,param_grid,DatasetName):
    for i in range(1,11):
        cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
        grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=cross_validation, scoring=f1,n_jobs=8)
        start = time.time()
        grid_result = grid_search.fit(x,y)
        end = time.time()
        knn_time=end-start
        random_state=i
        Knn_Para=grid_result.best_params_
        Knn_f1_score=grid_result.best_score_
        
        details = {
            'Random State':random_state,
            'name': fName,
            'time': knn_time,
            'f1_score':Knn_f1_score,
            'Best_Parameter':Knn_Para,
            
            
    
        }
        with open(DatasetName, 'a') as json_file:
            json.dump(details, json_file)
            json_file.write('\n')
     
        
        
    

In [ ]:
# increment ,function random state, time, best parameter

# k-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
f1 = make_scorer(f1_score , average='macro')

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_knn=cross_val_score(knn,x,y,cv=5,scoring=f1).mean()
end = time.time()
knn_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = KNeighborsClassifier()

n_neighbors = range(1, 21)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)

gridfun("KNN",x,y,model,grid,DatasetName)



# Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_linear=cross_val_score(knn,x,y,cv=5,scoring=f1).mean()
end = time.time()
Linear_dis_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = LinearDiscriminantAnalysis()

solver = ['svd', 'lsqr', 'eigen']
grid = dict(solver=solver)

gridfun("KNN",x,y,model,grid,DatasetName)

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
f1 = make_scorer(f1_score , average='macro')

In [ ]:
nb.priors

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_naive=cross_val_score(nb,x,y,cv=5,scoring=f1).mean()
end = time.time()
Naive_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = GaussianNB()


var_smoothing = [1e-09,2e-09,3e-09,4e-09,5e-09,6e-09,7e-09,8e-09,9e-09,10e-09]

grid = dict(var_smoothing=var_smoothing)

gridfun("Naive",x,y,model,grid,DatasetName)



# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm=SVC()
f1 = make_scorer(f1_score , average='macro')

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_support=cross_val_score(svm,x,y,cv=5,scoring=f1).mean()
end = time.time()
Support_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = SVC()

c = range(1, 20, 1)
kernel = ['poly', 'rbf', 'sigmoid']
gamma = ['scale', 'auto']
grid = dict(C=c,kernel=kernel,gamma=gamma)

gridfun("Support",x,y,model,grid,DatasetName)


# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=1)
f1 = make_scorer(f1_score , average='macro')

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_Logistic=cross_val_score(svm,x,y,cv=5,scoring=f1).mean()

end = time.time()
logistic_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = LogisticRegression()

# dual = ['bool','False']
penalty = [ 'l2']
# class_weight = ['dict', 'balanced']
grid = dict(penalty=penalty)

gridfun("Logistic",x,y,model,grid,DatasetName)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
f1 = make_scorer(f1_score , average='macro')

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_random=cross_val_score(rf,x,y,cv=5,scoring=f1).mean()

end = time.time()
random_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model=RandomForestClassifier()

n_estimators = range(1, 20, 1)
criterion = ['gini', 'entropy']
grid = dict(n_estimators=n_estimators,criterion=criterion)

gridfun("Random",x,y,model,grid,DatasetName)


# Ada boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
f1 = make_scorer(f1_score , average='macro')

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_Adaboost=cross_val_score(clf,x,y,cv=5,scoring=f1).mean()
end = time.time()
Adaboost_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = AdaBoostClassifier()

n_estimators = range(1, 100, 1)
algorithm = ['SAMME', 'SAMME.R']
grid = dict(n_estimators=n_estimators,algorithm=algorithm)

gridfun("Ada boost",x,y,model,grid,DatasetName)


# Gradient boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0)
f1 = make_scorer(f1_score , average='macro')   

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_gradient=cross_val_score(clf,x,y,cv=5,scoring=f1).mean()
end = time.time()
gradient_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model=GradientBoostingClassifier()

max_depth=range(1, 10, 1)
n_estimators=[100]
# learning_rate =np.arange(0.05, 0.2, 0.05)
learning_rate=[0.01]
grid = dict(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate)

gridfun("Gradient",x,y,model,grid,DatasetName)

# XGBoost

In [ ]:
import xgboost as xgb
gb_clf = xgb.XGBClassifier(eta=0.3,gamma=0,max_depth=6)
f1 = make_scorer(f1_score , average='macro')  

In [ ]:
from sklearn.model_selection import cross_val_score
start = time.time()
cross_val_score_xgboost=cross_val_score(gb_clf,x,y,cv=5,scoring=f1).mean()
end = time.time()
xgboost_time=end-start

In [ ]:
cross_validation = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model=xgb.XGBClassifier()

gamma=range(1, 10, 1)
eta=[0.001,0.01,0.05]
grid = dict(eta=eta,gamma=gamma)

gridfun("XGBoost",x,y,model,grid,DatasetName)

# save all croos validations

In [ ]:
details = {
    'Knn':cross_val_score_knn ,
    'Linear': cross_val_score_linear,
    'Naive':cross_val_score_naive,
    'support':cross_val_score_support,
    'Logistic':cross_val_score_Logistic,
    'Random':cross_val_score_random,
    'Ada boost':cross_val_score_Adaboost,
    'Gradient':cross_val_score_gradient,
    'XG Boost':cross_val_score_xgboost,
    
    
}
with open(cross, 'a') as json_file:
    json.dump(details, json_file)